<a href="https://colab.research.google.com/github/codedyasai/Python_MachineLearning/blob/main/06_%EC%9D%98_%EC%8B%A4%EC%8A%B5_iris.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Module Import

In [ ]:
from IPython.display import display
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
!pip install mglearn
import mglearn

# 음수표현 라이브러리
plt.rcParams['axes.unicode_minus'] = False

# 경고무시
import warnings
warnings.filterwarnings("ignore")

# 매직명령어 : 시각화 결과가 노트북에 포함되도록
%matplotlib inline

# 문제 정의 및 데이터셋 로딩
iris 쓸거임.

In [ ]:
iris = pd.read_csv('Iris.csv')

In [ ]:
iris.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             150 non-null    int64  
 1   SepalLengthCm  150 non-null    float64
 2   SepalWidthCm   150 non-null    float64
 3   PetalLengthCm  150 non-null    float64
 4   PetalWidthCm   150 non-null    float64
 5   Species        150 non-null    object 
dtypes: float64(4), int64(1), object(1)
memory usage: 7.2+ KB


In [ ]:
iris.describe()
# 스케일링을 해줘야 한다.

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm
count,150.000000,150.000000,150.000000,150.000000,150.000000
mean,75.500000,5.843333,3.054000,3.758667,1.198667
std,43.445368,0.828066,0.433594,1.764420,0.763161
min,1.000000,4.300000,2.000000,1.000000,0.100000
25%,38.250000,5.100000,2.800000,1.600000,0.300000
50%,75.500000,5.800000,3.000000,4.350000,1.300000
75%,112.750000,6.400000,3.300000,5.100000,1.800000
max,150.000000,7.900000,4.400000,6.900000,2.500000


In [ ]:
iris['Species'].unique()

array(['Iris-setosa', 'Iris-versicolor', 'Iris-virginica'], dtype=object)

In [ ]:
# 우리가 분류하려는 클래스의 0과 1의 수가 불균형한지 검사하자.
iris['Species'].value_counts()
# 차이가 없다.

Iris-setosa        50
Iris-versicolor    50
Iris-virginica     50
Name: Species, dtype: int64

In [ ]:
iris.columns

Index(['Id', 'SepalLengthCm', 'SepalWidthCm', 'PetalLengthCm', 'PetalWidthCm',
       'Species'],
      dtype='object')

In [ ]:
# 분석에 용이하게 Species를 숫자형으로 바꾼다.

mapping = {'Iris-setosa': 0, 'Iris-versicolor': 1, 'Iris-virginica': 2}
iris['Species'] = iris['Species'].map(mapping)

In [ ]:
iris_input = iris[['SepalLengthCm', 'SepalWidthCm', 'PetalLengthCm', 'PetalWidthCm']].to_numpy() # 사이킷런으로 작업하려면 array로 바꿔줘야 한다.
iris_target = iris[['Species']].to_numpy()

In [ ]:
iris_input.shape, iris_target.shape

((150, 4), (150, 1))

# 데이터 탐색 및 전처리

## 피처 스케일링

In [ ]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

input_scaler = StandardScaler().fit_transform(iris_input)

## 훈련세트와 테스트세트 분리

In [ ]:
from sklearn.model_selection import train_test_split

train_input, test_input, train_target, test_target, = train_test_split(
    input_scaler, iris_target, test_size= 0.2 , random_state=42)

# K-최근접 이웃 분류기 모델 구축

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier()
knn.fit(train_input, train_target)

KNeighborsClassifier()

In [ ]:
# 훈련데이터의 결정계수를 추출
knn.score(train_input, train_target)

0.9583333333333334

In [ ]:
knn.score(test_input, test_target)

1.0

- 테스트 데이터 점수가 더 높으니 **과소적합**이다.
- 모델을 더 복잡하게 만들자.

In [ ]:
# 모델을 복잡하게 만들기 위해 이웃수를 3으로 줄인다.
knn.n_neighbors = 3

# 모델을 다시 학습시킨다.
knn.fit(train_input, train_target)


KNeighborsClassifier(n_neighbors=3)

In [ ]:
knn.score(train_input, train_target)

0.9416666666666667

In [ ]:
knn.score(test_input, test_target)

1.0

- ? 더 떨어졌다. 기존의 이웃수=5가 더 나은듯.

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score
from sklearn.metrics import f1_score, confusion_matrix, roc_curve, precision_recall_curve

In [ ]:
pred = knn.predict(test_input)
pred_proba = knn.predict_proba(test_input)[:, 1]

In [ ]:
# get(실제값, 예측값, 예측확률)
def get(test_target, pred=None, pred_proba=None):
    confusion = confusion_matrix(test_target, pred)
    accuracy = accuracy_score(test_target, pred)
    precision = precision_score(test_target, pred, average='macro')
    recall = recall_score(test_target, pred, average='macro')
    f1 = f1_score(test_target, pred, average='macro')

    print('오차 행렬(혼돈 행렬)')
    print(confusion)

    print(f'정확도:{accuracy:.4f}, 정밀도:{precision:.4f}, 재현율:{recall:.4f}, F1:{f1:.4f}')

In [ ]:
get(test_target, pred)

오차 행렬(혼돈 행렬)
[[10  0  0]
 [ 0  9  0]
 [ 0  0 11]]
정확도:1.0000, 정밀도:1.0000, 재현율:1.0000, F1:1.0000


# 로지스틱 회귀 모델 구축

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
lr = LogisticRegression()
lr.fit(train_input, train_target)
pred = lr.predict(test_input)
pred_proba = lr.predict_proba(test_input)[:, 1]

In [ ]:
print(lr.score(train_input, train_target))
print(lr.score(test_input, test_target))

0.9666666666666667
1.0


In [ ]:
get(test_target, pred, pred_proba)

오차 행렬(혼돈 행렬)
[[10  0  0]
 [ 0  9  0]
 [ 0  0 11]]
정확도:1.0000, 정밀도:1.0000, 재현율:1.0000, F1:1.0000


In [ ]:
print(lr.coef_, lr.intercept_)

[[-1.0205419   1.12728796 -1.81774653 -1.69881636]
 [ 0.53378785 -0.28100139 -0.34450472 -0.72279041]
 [ 0.48675406 -0.84628656  2.16225125  2.42160678]] [-0.25413444  1.97519153 -1.72105709]


# 의사결정나무(Decision Tree)


In [ ]:
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier(random_state= 42)
dt.fit(train_input, train_target)

print(dt.score(train_input, train_target))
print(dt.score(test_input, test_target))

1.0
1.0


In [ ]:
# 특성중요도: 결정트리가 어떤 특성을 가장 중요하게 사용하였는지를 나타냄
dt.feature_importances_

array([0.        , 0.01667014, 0.90614339, 0.07718647])

NameError: ignored